In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [5]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

# Occasions Annotations

## Bin of 1 `guest_count`
Let's look at a few tables:
- `hockey_3_text_processed.csv`
- `silvester_3_text_processed.csv`
- `valentine_3_text_processed.csv`

to get an idea of the different occasions.

### Our Final set of **occasions** for the category 1 bin is (after iterations): 
**`Lunch`**
	- 1 Meal at least
	- 1 Drink
		○ More common no alcohol
**`Dinner`**
- Time label
    - `Late`
- 1 Meal at least
- 1 Drink

**`Drinking`**
- Time label
    - `Afternoon`
    - `Dinner`
    - `Late`
- More drink than food
    - Ratio of 3+:1+
- But more common no food at all
- More meal steps (because of the drinks)
- Longer dwell time
- If less drinks, but hard liquer, or more drinks but beer

**`Casual Drink & meal`**
- Time label
    - `Afternoon`
    - `Dinner`
    - `Late`
- Usually starts with drink/beer
- More drinks than food
    - Ratio of about 2:1
- Food might not be ordered in first step
- Maybe even last step


### Something to think about:
- `Lunch` vs `dinner`
- `how long do singles stay at a table`
- `alcohol versus beer`
- `food versus drinks`

----------

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 50)

In [6]:
hockey_path = '../data/hockey_3_text_processed.csv'
silvester_path = '../data/silvester_3_text_processed.csv'
valentine_path = '../data/valentine_3_text_processed.csv'

In [7]:
hockey = pd.read_csv(hockey_path)
silvester = pd.read_csv(silvester_path)
valentine = pd.read_csv(valentine_path)

Fix `datetime` features:

In [8]:
def fix_times(df):
    df['order_time'] = pd.to_datetime(df['order_time'], format="%Y-%m-%d %H:%M:%S.%f")
    df['order_time_closed'] = pd.to_datetime(df['order_time_closed'], format="%Y-%m-%d %H:%M:%S.%f")
    df['order_item_time'] = pd.to_datetime(df['order_item_time'], format="%Y-%m-%d %H:%M:%S.%f")

In [9]:
fix_times(hockey)
fix_times(valentine)
fix_times(silvester)

In [15]:
hockey.order_id.value_counts().keys()

Int64Index([521095549, 519477313, 514736044, 511415825, 520250633, 511378511,
            512851855, 516620908, 521100233, 525661404,
            ...
            520105238, 519527805, 514495361, 524862299, 514693984, 523514709,
            513545041, 512578876, 521098041, 511500325],
           dtype='int64', length=118664)

In [12]:
hockey = hockey[hockey.guest_count == 1]
valentine = valentine[valentine.guest_count == 1]
silvester = silvester[silvester.guest_count == 1]



In [13]:
def total_meal_steps(data):
    orders = {}
    order_ids = list(data.order_id.value_counts().keys())

    for order in order_ids:
        orders[order] = {}
            
        df = data[data.order_id == order]
        df = df.sort_values(by="order_item_time")
        last_meal_step = list(df.meal_step.sort_values())[-1]

        orders[order]['total_meal_steps'] = last_meal_step

        # from sit down to first order
        meal_flows = [(i * 4 + 1) for i in list(df.meal_flow_step)]

        orders[order]['sit_to_order'] = meal_flows[0]
        orders[order]['first_to_second_order'] = 0
        if len(meal_flows) > 1:
            orders[order]['first_to_second_order'] = meal_flows[1] - meal_flows[0]

        total_diff_flows = []
        prev_flow = 0
        for flow in meal_flows:
            if flow == prev_flow:
                continue
            total_diff_flows.append(flow - prev_flow)
            prev_flow = flow

        # avg time between orders
        avg_steps = sum(total_diff_flows)/len(total_diff_flows)
        orders[order]['avg_time_between_steps'] = avg_steps

    return orders

order_feats = total_meal_steps(hockey)
hockey['total_meal_steps'] = hockey.order_id.apply(lambda x:order_feats[x]['total_meal_steps'])
hockey['first_to_second_order'] = hockey.order_id.apply(lambda x:order_feats[x]['first_to_second_order'])
hockey['avg_time_between_steps'] = hockey.order_id.apply(lambda x:order_feats[x]['avg_time_between_steps'])

order_feats = total_meal_steps(valentine)
valentine['total_meal_steps'] = valentine.order_id.apply(lambda x:order_feats[x]['total_meal_steps'])
valentine['first_to_second_order'] = valentine.order_id.apply(lambda x:order_feats[x]['first_to_second_order'])
valentine['avg_time_between_steps'] = valentine.order_id.apply(lambda x:order_feats[x]['avg_time_between_steps'])

order_feats = total_meal_steps(silvester)
silvester['total_meal_steps'] = silvester.order_id.apply(lambda x:order_feats[x]['total_meal_steps'])
silvester['first_to_second_order'] = silvester.order_id.apply(lambda x:order_feats[x]['first_to_second_order'])
silvester['avg_time_between_steps'] = silvester.order_id.apply(lambda x:order_feats[x]['avg_time_between_steps'])


In [37]:
len(order_feats)

1

Lets only look at interesting features

In [14]:
features = ['order_id', 'order_time', 'order_time_closed',
       'order_item_time', 'title', 'category_id', 
       'dwell_time', 'meal_step',
       'meal_flow_last_to_close', 'total_flow_steps', 'meal_flow_step', 
       'beer_volume', 'order_day_of_week',
       'is_weekend', 'guest_count',
       'period_of_day', 
       'sharable', 'kids_meal', 'birthday',
       'total_sales_before_tax', 'item_qty', 'sales_before_tax']

In [15]:
hockey = hockey[features]
silvester = silvester[features]
valentine = valentine[features]

In [222]:
hockey.head()

,order_id,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
0,520008106,2019-04-16 12:01:00,2019-04-16 13:15:52,2019-04-16 12:13:26,$Sub.Swt.Pot.Fry-,2.0,1.247778,1,15,18,3,0.000,Tuesday,False,1,lunch,0,0,0,19.84,1.0,2.97
1,520008106,2019-04-16 12:01:00,2019-04-16 13:15:52,2019-04-16 12:13:26,FINGERS.SAUCED,2.0,1.247778,1,15,18,3,0.000,Tuesday,False,1,lunch,1,0,0,19.84,1.0,16.87
2,520008105,2019-04-16 12:01:00,2019-04-16 13:11:55,2019-04-16 12:18:25,TRAD.GUAC,2.0,1.181944,2,13,17,4,0.000,Tuesday,False,1,lunch,1,0,0,26.61,1.0,8.99
3,520008105,2019-04-16 12:01:00,2019-04-16 13:11:55,2019-04-16 12:13:26,VEGGIE.BURGER,2.0,1.181944,1,13,17,3,0.000,Tuesday,False,1,lunch,0,0,0,26.61,1.0,17.62
4,520253565,2019-04-16 19:35:00,2019-04-16 22:39:18,2019-04-16 20:56:13,CANADIAN.Pt,1.0,3.071667,3,12,44,19,0.473,Tuesday,False,0,dinner,0,0,0,30.48,1.0,7.62


Create table for each order, we're going to pick (randomly) 10 tables out of each dataframe.

In [16]:
def create_tables(df):
    tables = df.drop_duplicates(subset='order_id', keep='first')
    tables.set_index('order_id', inplace=True)
    tables = tables[['guest_count']]

    return tables

In [17]:
hockey_tables = create_tables(hockey)
valentine_tables = create_tables(valentine)
silvester_tables = create_tables(silvester)

In [18]:
print("We have ", len(hockey_tables), " tables in the Hockey data")

We have  49664  tables in the Hockey data


In [19]:
print("We have ", len(valentine_tables), " tables in the Valentine data")

We have  15485  tables in the Valentine data


In [20]:
print("We have ", len(silvester_tables), " tables in the Silvester data")

We have  35027  tables in the Silvester data


In [21]:
hockey.set_index('order_id', inplace=True)
valentine.set_index('order_id', inplace=True)
silvester.set_index('order_id', inplace=True)

-----

# The Annotations

Pick 10 tables out of each dataframe.

In [103]:
COUNT = 10

In [23]:
import random

In [237]:
def get_tables_gc_1(tables):
    return tables[tables['guest_count'] == 1]

## Annotating Hockey

Pick 10 tables of a total of 1 guests of **Hockey**.

In [127]:
hockey_1gc_tables = get_tables_gc_1(hockey_tables)
print("We have {} tables of a total of 1 guests".format(len(hockey_1gc_tables)))

We have 49664 tables of a total of 1 guests


In [128]:
#picked_hockey_tables = random.sample(list(hockey_1gc_tables.index), COUNT)

In [183]:
picked_hockey_tables = [512690383,
 521702519,
 521093892,
 517505343,
 521783372,
 521769692,
 512852707,
 525538989,
 514327163,
 520255453]

## Creating TEST Set

In [26]:
picked_hockey_tables_test = random.sample(list(hockey_tables.index), 10)
picked_hockey_tables_test

[520256354,
 514474566,
 522554624,
 522777463,
 519619252,
 519598706,
 512856854,
 520171895,
 520960843,
 520336377]

In [34]:
test_h = [520256354,
 514474566,
 522554624,
 522777463,
 519619252,
 519598706,
 512856854,
 520171895,
 520960843,
 520336377]

# DEV Annotations

# Table 1

In [26]:
#512690383
hockey[hockey.index == 512690383].sort_values(by=['order_item_time'])

,bar_id,order_time,order_time_closed,order_item_time,order_item_id,item_qty,sales_before_tax,sales_inc_tax,guest_count,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,total_orders_category_id_1.0,total_orders_category_id_2.0,total_orders_category_id_3.0,total_orders_category_id_4.0,total_orders_category_id_5.0,total_orders_category_id_6.0,total_orders,order_day_of_week,is_weekend,sharable,kids_meal,birthday,order_hour,period_of_day,order_minute,order_item_minute,order_item_hour,order_close_minute,order_close_hour,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,total_sales_before_tax,total_sales_inc_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
512690383,17650,2019-04-11 17:10:00,2019-04-11 18:23:30,2019-04-11 17:10:00,51269038300,1.0,5.95,6.724694,1,QUESA (VEG),2.0,0,0,0.0,0,2,1.0,0,0,0,3,Thursday,False,0,0,0,17,afternoon,10,10,17,23,18,1.225,1,7,17,0,20.43,23.09
512690383,17650,2019-04-11 17:10:00,2019-04-11 18:23:30,2019-04-11 17:16:00,51269038301,1.0,11.50,12.997308,1,FORBES MARG DBL,3.0,0,0,0.0,0,2,1.0,0,0,0,3,Thursday,False,0,0,0,17,afternoon,10,16,17,23,18,1.225,2,7,17,1,20.43,23.09
512690383,17650,2019-04-11 17:10:00,2019-04-11 18:23:30,2019-04-11 17:52:00,51269038302,0.5,2.98,3.367998,1,MINI DONUTS,2.0,0,0,0.0,0,2,1.0,0,0,0,3,Thursday,False,1,0,0,17,afternoon,10,52,17,23,18,1.225,3,7,17,10,20.43,23.09


**April 11th (Thursday), 17:10 - 18:23, a total of 1.2 hours, 3 meal steps**

**Meal Steps Description:**
1. Start
  - Meal (not very heavy or big)
      - Quesa
2. Still Start (6 minutes after)
  - Drink
      - Margarhita
3. Dessert, last order (~40+ minutes later)
  - Dessert
      - mini donutes
      
**Total drinks**: 1 cocktail, **total meals**: 1 main dish, 1 dessert

**`after work meal`** / **Casual drink and Meal**

`afternoon`, `small_meal`, `drink`

**Explanation**: Because of the time and the size of the meal, to me it seems like what a person after work would go and eat. Maybe even go to dinner after, but this seems like a "light" meal after work.

**Features that helped**: `type of meal` ("quesadia" is not really a main course), `price of meal` (5-6 $), and `time of meal` (5pm). 

# Table 2

In [163]:
hockey[hockey.index == 521702519].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_serving_type_id,beer_volume,total_orders,order_day_of_week,is_weekend,order_hour,period_of_day,order_item_minute,sharable,kids_meal,birthday,total_sales_before_tax,beer_brand_id,item_qty,sales_before_tax,sales_inc_tax,guest_count,total_sales_inc_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
521702519,2019-04-18 15:28:00,2019-04-18 16:15:18,2019-04-18 15:28:00,HENDRICKS,3.0,0.788333,1,7,11,0,0,0.0,2,Thursday,False,15,afternoon,28,0,0,0,18.5,0,1.0,9.25,10.455,1,20.91
521702519,2019-04-18 15:28:00,2019-04-18 16:15:18,2019-04-18 15:47:00,HENDRICKS,3.0,0.788333,2,7,11,4,0,0.0,2,Thursday,False,15,afternoon,47,0,0,0,18.5,0,1.0,9.25,10.455,1,20.91


**April 18th (Thursday), 15:28 - 16:15, total 0.7 hours, 2 meal steps**

**Meal Steps Description:**
1. Start
   - Drink
       - Gin
2. 20 Minutes Later:
   - Drink
       - Gin
       
**Total drinks**: 2 liquer drinks

**`Afternoon Drinking`**

`hard_liquer`, `afternoon`, `dwell_time`

**Explanation**: This is actually a very consistent person, he ordered two drinks exactly 20 minutes apart, and the table was closed ~20 minutes after the second drink.

**Features**: `time of meal` (3pm), `type of drink` (category 3, gin - hard liquer), `dwell time` (stayed ~40 min).

# Table 3

In [182]:
#521093892
hockey[hockey.index == 521093892].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
521093892,2019-04-17 22:04:00,2019-04-18 00:25:25,2019-04-17 22:42:29,MOSCOW MULE DBL,3.0,2.356944,1,0,33,9,0.0,Wednesday,False,1,hang_out,0,0,0,51.01,1.00,15.75
521093892,2019-04-17 22:04:00,2019-04-18 00:25:25,2019-04-17 23:17:10,MOSCOW MULE DBL,3.0,2.356944,2,0,33,17,0.0,Wednesday,False,1,hang_out,0,0,0,51.01,1.00,15.75
521093892,2019-04-17 22:04:00,2019-04-18 00:25:25,2019-04-18 00:04:36,VODKA RED BULL,3.0,2.356944,3,0,33,28,0.0,Wednesday,False,1,hang_out,0,0,0,51.01,1.00,9.50
521093892,2019-04-17 22:04:00,2019-04-18 00:25:25,2019-04-18 00:25:25,INCEPTION BOMB,3.0,2.356944,4,0,33,33,0.0,Wednesday,False,1,hang_out,0,0,0,51.01,0.28,4.06
521093892,2019-04-17 22:04:00,2019-04-18 00:25:25,2019-04-18 00:25:25,FIREBALL,3.0,2.356944,4,0,33,33,0.0,Wednesday,False,1,hang_out,0,0,0,51.01,1.00,5.95


**April 17th (Wednesday), 22:04 - 00:25, total of 2.35 hours, 4 meal steps.**

**Meal Steps Description:**
1. Start (40 minutes after sit down)
    - Drink
        - Moscow Mule (cocktail type drink)
2. 35 minutes after
    - Drink
        - Moscow Mule
3. 45 minutes after (1.15 hrs since person started drinking). (Notice the change to harsher liquer)
    - Drink
        - Vodka redbull (less of a sit down bar, more of a "dance" bar, or "Getting drunk bar", as opposed to previous drinks)
4. 20 minutes after that (~2 hours since started drinking)
    - Drink
        - Inception Bomb (another weird whisky shot)
        - Fireball (sweet whisky shot)
     (two for the road?)

**Total drinks: 2 cocktails, 1 mixed drink, 2 shots**

**`drinking`**

`late_night`, `hard_liquer`, `quantity of drinks`, `dwell_time`

**Explanation**: Its hard to say if this person was watching a game or something, or was ordering at a bar and sitting with a group. I would guess that the person is by themselves, since its a bit weird to keep ordering single drinks for yourself the whole night, even shots? Although at the end the person orders two whisky shots... hmm. Still I would say a single person drinking at night, could be more of a dance bar because of the type of drinks.
(its interesting also to note that the person started with classy drinks, and ended with whisky :P)

**Features**: `time of the order` and `closing time` (started at 10pm and ended past midnight), `quantity of drinks` (had a total of 5 drinks)

# Table 4

In [185]:
hockey[hockey.index == 524421347].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
524421347,2019-04-21 17:31:00,2019-04-21 18:39:40,2019-04-21 17:31:04,WINGS.10PC,2.0,1.144444,1,10,16,0,0.0,Sunday,True,3,afternoon,1,0,0,89.35,1.0,16.99
524421347,2019-04-21 17:31:00,2019-04-21 18:39:40,2019-04-21 17:31:04,KIDPOP,4.0,1.144444,1,10,16,0,0.0,Sunday,True,3,afternoon,0,0,0,89.35,1.0,0.00
524421347,2019-04-21 17:31:00,2019-04-21 18:39:40,2019-04-21 17:31:04,JACKS.VEG.NACHOS,2.0,1.144444,1,10,16,0,0.0,Sunday,True,3,afternoon,1,0,0,89.35,1.0,21.26
524421347,2019-04-21 17:31:00,2019-04-21 18:39:40,2019-04-21 17:31:04,WINGS,2.0,1.144444,1,10,16,0,0.0,Sunday,True,3,afternoon,1,0,0,89.35,1.0,0.00
524421347,2019-04-21 17:31:00,2019-04-21 18:39:40,2019-04-21 17:31:04,WINGS,2.0,1.144444,1,10,16,0,0.0,Sunday,True,3,afternoon,1,0,0,89.35,1.0,0.00
524421347,2019-04-21 17:31:00,2019-04-21 18:39:40,2019-04-21 17:31:04,KIDPOP,4.0,1.144444,1,10,16,0,0.0,Sunday,True,3,afternoon,0,0,0,89.35,1.0,0.00
524421347,2019-04-21 17:31:00,2019-04-21 18:39:40,2019-04-21 17:31:04,WINGS.10PC,2.0,1.144444,1,10,16,0,0.0,Sunday,True,3,afternoon,1,0,0,89.35,1.0,16.99
524421347,2019-04-21 17:31:00,2019-04-21 18:39:40,2019-04-21 17:31:04,POP.,4.0,1.144444,1,10,16,0,0.0,Sunday,True,3,afternoon,0,0,0,89.35,1.0,3.99
524421347,2019-04-21 17:31:00,2019-04-21 18:39:40,2019-04-21 17:37:39,STRW.MINTSODA,3.0,1.144444,2,10,16,1,0.0,Sunday,True,3,afternoon,0,0,0,89.35,1.0,8.56


** Mistake in the category **

We can add a rule for this

Notice there are only 3 meal steps but 7 food type items.
There is also a "kid" meal in here.

Here we can filter per meal type, there shouldn't be more than MAX 3 food items in one meal step. (if not total).

# Table 5

In [186]:
hockey[hockey.index == 521783372].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
521783372,2019-04-18 21:01:00,2019-04-18 21:20:02,2019-04-18 21:01:14,SMOKIN.SALMON,2.0,0.317222,1,4,4,0,0.0,Thursday,False,1,hang_out,0,0,0,23.96,1.0,23.96
521783372,2019-04-18 21:01:00,2019-04-18 21:20:02,2019-04-18 21:01:14,Salmon.Rice/Veg,2.0,0.317222,1,4,4,0,0.0,Thursday,False,1,hang_out,0,0,0,23.96,1.0,0.00


**April 18th (Thursday), 21:01 - 21:20, total of 0.3 hours, and 1 meal steps.**

**Meal Steps Description:**
1. Start 
    - Dinner (classic)
        - Salmon
        
**Total meals: 1 large/dinner meal**

**`dinner`**

`dwell_time`, `food_category_only`, `dinner_time`.

**Explanation**: Not much to explain, very classic single person dinner.


# Table 6

In [187]:
hockey[hockey.index == 521769692].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
521769692,2019-04-18 18:41:00,2019-04-18 20:44:24,2019-04-18 18:41:00,LRG BONESHAKER,1.0,2.056667,1,19,29,0,0.591,Thursday,False,1,dinner,0,0,0,29.5,1.0,7.25
521769692,2019-04-18 18:41:00,2019-04-18 20:44:24,2019-04-18 19:24:00,BUF CHIC SALAD,2.0,2.056667,2,19,29,10,0.000,Thursday,False,1,dinner,0,0,0,29.5,1.0,15.00
521769692,2019-04-18 18:41:00,2019-04-18 20:44:24,2019-04-18 19:25:00,LRG BONESHAKER,1.0,2.056667,2,19,29,10,0.591,Thursday,False,1,dinner,0,0,0,29.5,1.0,7.25


**April 18th (Thursday), 18:41 - 20:44, total of 2 hours, 2 meal steps.**

**Meal Steps Description:**

1. Start:
   - Drink
       - Half a liter of beer (ordered when sat down)
2. ~40 minutes after
   - Drink
       - Half a liter of beer (ordered when sat down)
   - Meal
       - Salad

**total drinks**: 2 half liter beers, total meals: 1 salad

**`casual drink & meal`** - (I guess similar to relaxing alone ?)

`dwell_time`, `beer`, `time_between_drinks`, `dinner_time`

**Explanation**: I didn't want to say casual dinner, because the person first ordered a beer, and drank the beer for 40 minutes. I would say that he was watching a game or something, and after 40 minutes (maybe half time?) ordered food and another drink. Especially since he then goes on to sit another 1.2 hours with a beer and salad.. I mean no one takes that long to eat a salad unless they're donig someting else.

**Features**:  The fact that there was a long period of time for two beers. The time it took between beers, the time of the table, dinner time, and the light meal.



# Table 7

In [188]:
hockey[hockey.index == 512852707].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
512852707,2019-04-11 23:40:00,2019-04-12 01:01:14,2019-04-11 23:40:03,GUINNESS.Pt,1.0,1.353889,1,0,20,0,0.562,Thursday,False,1,night,0,0,0,27.57,1.0,8.82
512852707,2019-04-11 23:40:00,2019-04-12 01:01:14,2019-04-12 00:41:03,GUINNESS.Pt,1.0,1.353889,2,0,20,15,0.562,Thursday,False,1,night,0,0,0,27.57,1.0,8.82
512852707,2019-04-11 23:40:00,2019-04-12 01:01:14,2019-04-12 01:01:14,$Sub.Swt.Pot.Fry-,2.0,1.353889,3,0,20,20,0.000,Thursday,False,1,night,0,0,0,27.57,0.5,1.49
512852707,2019-04-11 23:40:00,2019-04-12 01:01:14,2019-04-12 01:01:14,FINGERS.SAUCED,2.0,1.353889,3,0,20,20,0.000,Thursday,False,1,night,1,0,0,27.57,0.5,8.44


**April 11th (Thursday), 23:40 - 01:01, total of 1.35 hours, 3 meal steps.**

**Meal Steps Description:**
1. Start:
    - drink
        - Guinnes, half a liter
2. 1hr later
    - drink
        - Guiness, half a liter
3. 20 minutes later
    - meal
        - chicken fingers, and sweet potato fries
        
**total drinks**: 2 beers, **total meals**: 1 "sharable/late night" food.

`late casual drink & meal`

`late_night`, `beer`, `dwell_time`, `sharable_food`(bar food).

**Explanation**: Its a classic table, maybe also watchign a game or something. Starts with a beer and sits for an hour till person orders another drink. Then twenty minutes later orders food and closes the table.
Its a late night, the person is drinking beer, and eating bar type/sharable food. Similar to previous table.

**Features**: staying at the table for 2 hours, and having a long time between drinks hints at a casual drink/game watching table.

# Table 8

In [189]:
hockey[hockey.index == 525538989].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
525538989,2019-04-23 17:44:00,2019-04-23 19:25:47,2019-04-23 17:44:35,KEITHS.XL,1.0,1.696389,1,9,23,0,0.591,Tuesday,False,1,afternoon,0,0,0,39.99,1.0,10.01
525538989,2019-04-23 17:44:00,2019-04-23 19:25:47,2019-04-23 18:27:36,KEITHS.XL,1.0,1.696389,2,9,23,10,0.591,Tuesday,False,1,afternoon,0,0,0,39.99,1.0,10.01
525538989,2019-04-23 17:44:00,2019-04-23 19:25:47,2019-04-23 18:46:51,CHEESE.CHIKN.MAC,2.0,1.696389,3,9,23,14,0.000,Tuesday,False,1,afternoon,0,0,0,39.99,1.0,19.97


**April 23rd (Tuesday), 17:44 - 19:25, total of 1.7 hours, 3 meal steps.**

**Meal Steps Description:**
1. Start:
    - drink
        - Keiths, half a liter
2. 45 min later
    - drink
        - Keiths, half a liter
3. 20 minutes later
    - meal
        - chicken mac and cheese (bar food.. amarican bar food. I miss mac&cheese)
        
**total drinks**: 2 beers, **total meals**: 1 "sharable/late night" food.

**`casual drink & meal`**

`late_night`, `beer`, `dwell_time`, `sharable_food`(bar food).

**Explanation**: Its a classic table, maybe also watchign a game or something. Starts with a beer and sits for an hour till person orders another drink. Then twenty minutes later orders food and closes the table.
Its a late night, the person is drinking beer, and eating bar type/sharable food. Similar to previous table.

**Features**: staying at the table for 2 hours, and having a long time between drinks hints at a casual drink/game watching table.

# Hockey TEST annotations

In [35]:
test_h

[520256354,
 514474566,
 522554624,
 522777463,
 519619252,
 519598706,
 512856854,
 520171895,
 520960843,
 520336377]

### Table 1

In [29]:
hockey[hockey.index == 520256354].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
520256354,2019-04-16 19:46:00,2019-04-16 23:44:17,2019-04-16 19:46:00,POLAR ICE,3.0,3.971389,1,45,59,0,0.0,Tuesday,False,1,dinner,0,0,0,10.5,1.0,5.25
520256354,2019-04-16 19:46:00,2019-04-16 23:44:17,2019-04-16 20:42:00,POLAR ICE,3.0,3.971389,2,45,59,14,0.0,Tuesday,False,1,dinner,0,0,0,10.5,1.0,5.25


**->DRINKING**

### Table 2

In [36]:
hockey[hockey.index == 514474566].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
514474566,2019-04-13 15:42:00,2019-04-13 16:44:42,2019-04-13 15:42:51,PINAPL.MARG,3.0,1.045,1,12,14,0,0.0,Saturday,True,1,afternoon,0,0,0,32.96,1.0,14.99
514474566,2019-04-13 15:42:00,2019-04-13 16:44:42,2019-04-13 15:54:45,HOLY.SMOKE.BURGR,2.0,1.045,2,12,14,2,0.0,Saturday,True,1,afternoon,0,0,0,32.96,1.0,17.97


**->Casual drink and meal**

### Table 3

In [38]:
hockey[hockey.index == 522554624].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
522554624,2019-04-19 16:01:00,2019-04-19 17:12:31,2019-04-19 16:01:05,WATER.,4.0,1.191944,1,16,17,0,0.0,Friday,True,1,afternoon,0,0,0,46.86,1.00,0.00
522554624,2019-04-19 16:01:00,2019-04-19 17:12:31,2019-04-19 16:01:05,BLURASP LEMONADE,3.0,1.191944,1,16,17,0,0.0,Friday,True,1,afternoon,0,0,0,46.86,1.67,12.37
522554624,2019-04-19 16:01:00,2019-04-19 17:12:31,2019-04-19 16:01:05,WATER.,4.0,1.191944,1,16,17,0,0.0,Friday,True,1,afternoon,0,0,0,46.86,1.00,0.00
522554624,2019-04-19 16:01:00,2019-04-19 17:12:31,2019-04-19 16:01:05,ORNG.DREAMSCL,3.0,1.191944,1,16,17,0,0.0,Friday,True,1,afternoon,0,0,0,46.86,1.00,8.97
522554624,2019-04-19 16:01:00,2019-04-19 17:12:31,2019-04-19 16:07:25,POUT.CLASSIC,2.0,1.191944,2,16,17,1,0.0,Friday,True,1,afternoon,0,0,0,46.86,1.00,9.53
522554624,2019-04-19 16:01:00,2019-04-19 17:12:31,2019-04-19 16:07:25,FALAFEL-WICH,2.0,1.191944,2,16,17,1,0.0,Friday,True,1,afternoon,0,0,0,46.86,1.00,15.99


**->Not Category 1**

### Table 4

In [40]:
hockey[hockey.index == 522777463].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
522777463,2019-04-20 00:36:00,2019-04-20 01:00:08,2019-04-20 00:36:28,HEINEKEN,1.0,0.402222,1,0,5,0,0.33,Saturday,True,1,night,0,0,0,17.75,1.0,7.25
522777463,2019-04-20 00:36:00,2019-04-20 01:00:08,2019-04-20 00:42:24,CROWN ROYAL,3.0,0.402222,2,0,5,1,0.00,Saturday,True,1,night,0,0,0,17.75,1.0,7.25
522777463,2019-04-20 00:36:00,2019-04-20 01:00:08,2019-04-20 01:00:08,RED BULL STAFF,4.0,0.402222,3,0,5,5,0.00,Saturday,True,1,night,0,0,0,17.75,1.0,3.25


**->DRINKING**

### Table 5

In [42]:
hockey[hockey.index == 519619252].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
519619252,2019-04-15 19:06:00,2019-04-15 22:13:03,2019-04-15 19:06:17,BLOODY CAESAR,3.0,3.1175,1,21,46,0,0.000,Monday,False,1,dinner,0,0,0,27.0,1.0,9.0
519619252,2019-04-15 19:06:00,2019-04-15 22:13:03,2019-04-15 19:11:10,ONION RING,2.0,3.1175,2,21,46,1,0.000,Monday,False,1,dinner,0,0,0,27.0,1.0,3.0
519619252,2019-04-15 19:06:00,2019-04-15 22:13:03,2019-04-15 19:48:35,PNT BUDWEISER,1.0,3.1175,3,21,46,10,0.473,Monday,False,1,dinner,0,0,0,27.0,1.0,5.0
519619252,2019-04-15 19:06:00,2019-04-15 22:13:03,2019-04-15 20:48:53,Bite,5.0,3.1175,4,21,46,25,0.000,Monday,False,1,dinner,0,0,0,27.0,1.0,3.0
519619252,2019-04-15 19:06:00,2019-04-15 22:13:03,2019-04-15 20:48:53,BEER N BITE,1.0,3.1175,4,21,46,25,0.473,Monday,False,1,dinner,0,0,0,27.0,1.0,7.0


**->Casual drink and meal** - this one was hard, I would say its casual becase of the time between drinks

### Table 6

In [43]:
hockey[hockey.index == 519598706].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
519598706,2019-04-15 22:33:00,2019-04-15 23:09:00,2019-04-15 22:36:50,CHK.PARMIGIANA,2.0,0.6,1,8,8,0,0.0,Monday,False,1,hang_out,0,0,0,20.87,1.0,20.87


**->Dinner**

### Table 7

In [44]:
hockey[hockey.index == 512856854].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
512856854,2019-04-11 13:09:00,2019-04-11 22:29:51,2019-04-11 13:09:00,JUG CANADIAN,1.0,9.3475,1,119,139,0,0.4435,Thursday,False,1,lunch,0,0,0,33.75,0.25,5.25
512856854,2019-04-11 13:09:00,2019-04-11 22:29:51,2019-04-11 13:09:00,JUG CANADIAN,1.0,9.3475,1,119,139,0,0.4435,Thursday,False,1,lunch,0,0,0,33.75,0.25,5.25
512856854,2019-04-11 13:09:00,2019-04-11 22:29:51,2019-04-11 13:31:00,SHRIMP FAJITAS,2.0,9.3475,2,119,139,5,0.0000,Thursday,False,1,lunch,0,0,0,33.75,1.00,18.00
512856854,2019-04-11 13:09:00,2019-04-11 22:29:51,2019-04-11 14:31:00,JUG MILLSTREET,1.0,9.3475,3,119,139,20,0.4435,Thursday,False,1,lunch,0,0,0,33.75,0.25,5.25


**->Drinking** - weird one, who orders two half liters right off the bat?m

### Table 8

In [46]:
hockey[hockey.index == 520171895].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
520171895,2019-04-16 20:23:00,2019-04-16 21:29:43,2019-04-16 20:23:46,QUESADILLAS,2.0,1.111944,1,16,16,0,0.0,Tuesday,False,1,dinner,1,0,0,7.48,1.0,7.48


**->Dinner**

# Valentine

Pick 5 tables of total of 1 guests for **Valentine**.

In [199]:
valentine_1gc = get_tables_gc_1(valentine_tables)
print("We have {} tables of a total of 1 guests".format(len(valentine_1gc)))

We have 15485 tables of a total of 1 guests


In [200]:
picked_valentine_tables = random.sample(list(valentine_1gc.index), COUNT)

In [204]:
picked_valentine_tables = [434780854,
 434753224,
 447023765,
 434766693,
 446799837]

## Table 1

In [206]:
valentine[valentine.index == 434780854].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
434780854,2019-02-10 14:13:00,2019-02-10 17:18:03,2019-02-10 14:13:00,BEEFEATER**,3.0,3.084167,1,9,44,0,0.000,Sunday,True,1,afternoon,0,0,0,62.09,1.5,3.40
434780854,2019-02-10 14:13:00,2019-02-10 17:18:03,2019-02-10 14:13:00,MARTINI,3.0,3.084167,1,9,44,0,0.000,Sunday,True,1,afternoon,0,0,0,62.09,1.0,8.97
434780854,2019-02-10 14:13:00,2019-02-10 17:18:03,2019-02-10 14:13:00,WATER.,4.0,3.084167,1,9,44,0,0.000,Sunday,True,1,afternoon,0,0,0,62.09,1.0,0.00
434780854,2019-02-10 14:13:00,2019-02-10 17:18:03,2019-02-10 14:28:00,PAD THAI,2.0,3.084167,2,9,44,3,0.000,Sunday,True,1,afternoon,0,0,0,62.09,1.0,0.00
434780854,2019-02-10 14:13:00,2019-02-10 17:18:03,2019-02-10 14:28:00,BEEFEATER**,3.0,3.084167,2,9,44,3,0.000,Sunday,True,1,afternoon,0,0,0,62.09,1.5,3.40
434780854,2019-02-10 14:13:00,2019-02-10 17:18:03,2019-02-10 14:28:00,PAD.THAI.CHICKN,2.0,3.084167,2,9,44,3,0.000,Sunday,True,1,afternoon,0,0,0,62.09,1.0,20.74
434780854,2019-02-10 14:13:00,2019-02-10 17:18:03,2019-02-10 15:10:00,BAR.MANGER.Pt,1.0,3.084167,3,9,44,13,0.473,Sunday,True,1,afternoon,0,0,0,62.09,1.0,8.72
434780854,2019-02-10 14:13:00,2019-02-10 17:18:03,2019-02-10 15:44:00,RICKARD.RED.Pt,1.0,3.084167,4,9,44,21,0.473,Sunday,True,1,afternoon,0,0,0,62.09,1.0,8.43
434780854,2019-02-10 14:13:00,2019-02-10 17:18:03,2019-02-10 16:40:00,RICKARD.RED.Pt,1.0,3.084167,5,9,44,35,0.473,Sunday,True,1,afternoon,0,0,0,62.09,1.0,8.43


**Feb 10th (Sunday), 14:13 - 17:18, total of ~3 hour, 5 meal steps.**

**Meal Steps Description:**
1. Start:
    - drink
        - martini
        - Beefeater ??? -cheap, maybe shot?
        - water
2. 15 min later
    - meal
        - pad thai chicken
    - drink
        - beefeater ???
3. 40 min later
    - drink
        - "bad manager" beer, half liter
      
4. 35 min later
    - drink
        - red pt, half liter
5. 1 hour later
    - drink
        - red pt, hald liter
        

**total drinks**: 4 or 6, depending if we cound beefeater, **total meals**: 1 meal.

**`afternoon drink & meal`**

`dwell_time`, `amount of alcohol`, `time between drinks`

**Explanation**: This one was not easy. 3 whole hours at a bar. at first it looks like a mistake, but then you see that there are actual steps, where you start with a drink, eat a meal, and finish with two more drinks. Hard to say what type of occasion this is, but I would say a afternoon meal and drink (not casual?), since there is a meal consumed, and a good amount of drink and beer consumed.

- I wanted to say casual at first, but maybe because of the martini and amount of total alcohol its not casual?.. but its 2pm :/


# Table 2

In [207]:
valentine[valentine.index == 434753224].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
434753224,2019-02-10 21:56:00,2019-02-10 22:25:51,2019-02-10 21:56:00,WINGS..,2.0,0.4975,1,7,7,0,0.0,Sunday,True,1,hang_out,1,0,0,10.47,1.0,0.00
434753224,2019-02-10 21:56:00,2019-02-10 22:25:51,2019-02-10 21:56:00,WINGS.5PC,2.0,0.4975,1,7,7,0,0.0,Sunday,True,1,hang_out,1,0,0,10.47,1.0,10.47


**Feb 10th (Sunday), 21:56 - 22:25, total of 0.5 hours (30 minutes), 1 meal steps.**

**Meal Steps Description:**
1. Start:
    - meal
        - chicken fingers
        
**total meals**: 1 late night sharable meal

**`late night munch`** - a sharble item late at night

`sharable`, `late_night`, `1_step/1 item`


**Explanation**: Looks like a late munch. I would even say maybe its a take out, then again it could be a person stopped for wings and then left.



# Table 3

In [211]:
valentine[valentine.index == 447023765].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
447023765,2019-02-13 13:08:00,2019-02-13 14:07:08,2019-02-13 13:08:00,POP.,4.0,0.985556,1,4,14,0,0.0,Wednesday,False,1,lunch,0,0,0,23.07,1.0,3.39
447023765,2019-02-13 13:08:00,2019-02-13 14:07:08,2019-02-13 13:12:00,FINGERS.SAUCED,2.0,0.985556,2,4,14,1,0.0,Wednesday,False,1,lunch,1,0,0,23.07,1.0,14.34
447023765,2019-02-13 13:08:00,2019-02-13 14:07:08,2019-02-13 13:49:00,BROWNIE.COOKIE,2.0,0.985556,3,4,14,10,0.0,Wednesday,False,1,lunch,0,0,0,23.07,1.0,5.34


**Feb 13th (Wednesday), 13:08 - 14:07, total of ~1 hour, 3 meal steps.**

**Meal Steps Description:**
1. Start:
    - drink
        - soda pop
2. 4 min later
    - meal
        - chicken fingers
3. 35 minutes after
    - dessert
        - brownie

**total drinks**: 1 non alc drink, **total meals**: 1 lunch meal, and 1 dessert.

**`lunch`**

`lunch_time`, `meal`, `dwell_time`

**Explanation**: classic lunch, even went for dessert. Lunches shouldn't usually dwell too long. no alcohol and a single meal.

**Features**: time of meal (1pm), no alcohol, and spent about an hour.

# Table 4

In [209]:
valentine[valentine.index == 434766693].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
434766693,2019-02-10 00:50:00,2019-02-10 02:07:40,2019-02-10 00:50:00,PNT AMSTERDAM,1.0,1.294444,1,1,18,0,0.591,Sunday,True,1,night,0,0,0,13.72,1.0,6.86
434766693,2019-02-10 00:50:00,2019-02-10 02:07:40,2019-02-10 02:01:00,PNT MILL ST,1.0,1.294444,2,1,18,17,0.591,Sunday,True,1,night,0,0,0,13.72,1.0,6.86


**Feb 10th (Tuesday), 00:50 - 02:07, total of 1.3 hours, 2 meal steps.**

**Meal Steps Description:**
1. Start:
    - drink
        - amsterdam beer, half liter
2. 1 hr and 10 min later
    - drink
        - mills, half liter

**total drinks**: 2 beer drinks

**`late casual drink`** I think theres a difference between drinking late at night beer, and hard liquer.

    `time_between_drinks`, `beer`, `late_night` 

**Explanation**: pretty straight forward, late at night, drinking beer and taking its time. drinking a beer over an hour and then a second one.

**Features**: an hour between beers, having a total of two beers (seems like a common number of beers), and late night

#### 5. Table 446744761

In [210]:
valentine[valentine.index == 446799837].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
446799837,2019-02-12 18:37:00,2019-02-12 19:57:07,2019-02-12 18:37:00,R.JT.MERLOT.Gl,6.0,1.335278,1,17,19,0,0.0,Tuesday,False,1,dinner,0,0,0,36.71,1.0,6.00
446799837,2019-02-12 18:37:00,2019-02-12 19:57:07,2019-02-12 18:46:00,BACON.FILET*,2.0,1.335278,2,17,19,2,0.0,Tuesday,False,1,dinner,0,0,0,36.71,1.0,30.71


**Feb 12th (Tuesday), 18:37 - 19:57, total of 1.3 hours, 2 meal steps.**

**Meal Steps Description:**
1. Start:
    - drink
        - Wine
2. 10 min later
    - meal
        - Bacon Filet (huh?.. fancy)

**total drinks**: 1 drink, **total meals**: 1 dinner meal.

**`dinner`** (maybe even a celebratory dinner, or "pinuk" dinner? "treat yo self" type dinner.

`wine`, `expensive_meal`, `dinner_time`

**Explanation**: classic dinner table, although important to note that the wine and filet makes it a bit more fancy, special occasion type:)

**Features**: expensive meal -(30$), dinner time (6pm), `wine`.

--------------

# Silvester

In [239]:
silvester_1gc = get_tables_gc_1(silvester_tables)
print("We have {} tables of a total of 1 guests".format(len(silvester_1gc)))

We have 35027 tables of a total of 1 guests


In [244]:
#picked_silvester_tables = random.sample(list(silvester_1gc.index), COUNT)

In [246]:
picked_silvester_tables = [363436344,
 361891515,
 361024418,
 360026487,
 359644172]

# Table 1

In [247]:
silvester[silvester.index == 363436344].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
363436344,2019-01-03 17:39:00,2019-01-04 00:01:01,2019-01-03 17:39:00,VEGGIE.BURGER,2.0,6.366944,1,95,95,0,0.0,Thursday,False,1,afternoon,0,0,0,8.81,1.0,8.81


**Jan 03rd (Thursday), 17:39 - 00:01, total of 6 hours (mistake), 1 meal steps.**

**Meal Steps Description:**
1. Start:
    - meal
        - veggie burger


**total meals**: 1 dinner meal.

**`dinner`** 

`dinner_time`, `1_meal`

**Explanation**: classic dinner for 1. Only mistake I see is they didnt close the table, and so the dwell time says 6 hours.


# Table 2

In [248]:
silvester[silvester.index == 361891515].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
361891515,2018-12-30 22:47:00,2018-12-30 23:45:32,2018-12-30 22:47:00,V.SASSAFRAZ,4.0,0.975556,1,14,14,0,0.000,Sunday,True,1,hang_out,0,0,0,45.24,1.0,4.95
361891515,2018-12-30 22:47:00,2018-12-30 23:45:32,2018-12-30 22:47:00,GUINNESS.Pt,1.0,0.975556,1,14,14,0,0.473,Sunday,True,1,hang_out,0,0,0,45.24,1.0,9.33
361891515,2018-12-30 22:47:00,2018-12-30 23:45:32,2018-12-30 22:47:00,DRUNKEN.PLATTER,2.0,0.975556,1,14,14,0,0.000,Sunday,True,1,hang_out,0,0,0,45.24,1.0,29.97
361891515,2018-12-30 22:47:00,2018-12-30 23:45:32,2018-12-30 22:48:00,Loaded.Mash,2.0,0.975556,1,14,14,0,0.000,Sunday,True,1,hang_out,0,0,0,45.24,1.0,0.99


**Dec 30th (Sunday), 22:47 - 23:45, 0.9 hrs, 1 meal steps.**

**Meal Steps Description:**
1. Start:
    - drink
        - non alcoholic, sassafraz??
        - guinness, half liter
    - meal
        - Drunken platter, 29\$
        - loaded mash, 0.9$


**Total drinks**: 1 half liter, **total meals**: 1 large meal

**`Late dinner`** 

`late night`, `1 drink`, `1 meals`, `1 expensive meal`

**Explanation**: Another classic dinner for 1, but late at night.


# Table 3

In [250]:
silvester[silvester.index == 361024418].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
361024418,2018-12-28 22:17:00,2018-12-28 23:13:13,2018-12-28 22:17:00,STRAW.PALOMA,3.0,0.936944,1,13,13,0,0.0,Friday,True,1,hang_out,0,0,0,29.98,1.0,13.99
361024418,2018-12-28 22:17:00,2018-12-28 23:13:13,2018-12-28 22:18:00,CHK.GOATCHS.WRAP,2.0,0.936944,1,13,13,0,0.0,Friday,True,1,hang_out,0,0,0,29.98,1.0,15.99


**Dec 28th (Friday), 22:17 - 23:13, 0.9 hrs, 1 meal steps.**

**Meal Steps Description:**
1. Start:
    - drink
        - cocktail, strawberry paloma
    - meal
        - chicken wrap


**Total drinks**: 1 cocktail, **total meals**: 1 meal

**`Late dinner`** 

`late night`, `1 drink`, `1 meal`, 

**Explanation**: Another classic dinner for 1, but late at night.


# Table 4

In [251]:
silvester[silvester.index == 360026487].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
360026487,2018-12-26 17:42:00,2018-12-26 18:15:35,2018-12-26 17:42:00,FAJ.TWO.CHK.SHRMP,2.0,0.559722,1,8,8,0,0.0,Wednesday,False,1,afternoon,0,0,0,36.97,1.0,36.97
360026487,2018-12-26 17:42:00,2018-12-26 18:15:35,2018-12-26 17:42:00,FAJ.FOR.TWO,2.0,0.559722,1,8,8,0,0.0,Wednesday,False,1,afternoon,0,0,0,36.97,1.0,0.00


**Dec 26th (Wednesday), 17:42 - 18:15, 0.55 hrs, 1 meal steps.**

**Meal Steps Description:**
1. Start:
    - meal
        - fajitas chicken shrimp (big meal)


**total meals**: 1 meal

**`dinner`** 

`dinner time`, `1 meal`,`big meal` 

**Explanation**: classic dinner for 1, a lot of food.


# Table 5

In [252]:
silvester[silvester.index == 359644172].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
359644172,2018-12-25 18:18:00,2018-12-25 22:18:55,2018-12-25 18:18:00,KEITH'S TAP,1.0,4.015278,1,20,58,0,0.591,Tuesday,False,1,dinner,0,0,0,55.1,1.0,6.5
359644172,2018-12-25 18:18:00,2018-12-25 22:18:55,2018-12-25 18:18:00,POTATO SKINS,2.0,4.015278,1,20,58,0,0.000,Tuesday,False,1,dinner,0,0,0,55.1,1.0,8.2
359644172,2018-12-25 18:18:00,2018-12-25 22:18:55,2018-12-25 18:43:00,KEITH'S TAP,1.0,4.015278,2,20,58,6,0.591,Tuesday,False,1,dinner,0,0,0,55.1,1.0,6.5
359644172,2018-12-25 18:18:00,2018-12-25 22:18:55,2018-12-25 19:02:00,KEITH'S TAP,1.0,4.015278,3,20,58,10,0.591,Tuesday,False,1,dinner,0,0,0,55.1,1.0,6.5
359644172,2018-12-25 18:18:00,2018-12-25 22:18:55,2018-12-25 19:03:00,PEROGIES,2.0,4.015278,3,20,58,10,0.000,Tuesday,False,1,dinner,0,0,0,55.1,1.0,7.9
359644172,2018-12-25 18:18:00,2018-12-25 22:18:55,2018-12-25 19:59:00,KEITH'S TAP,1.0,4.015278,4,20,58,24,0.591,Tuesday,False,1,dinner,0,0,0,55.1,1.0,6.5
359644172,2018-12-25 18:18:00,2018-12-25 22:18:55,2018-12-25 20:23:00,KEITH'S TAP,1.0,4.015278,5,20,58,30,0.591,Tuesday,False,1,dinner,0,0,0,55.1,1.0,6.5
359644172,2018-12-25 18:18:00,2018-12-25 22:18:55,2018-12-25 20:57:00,KEITH'S TAP,1.0,4.015278,6,20,58,38,0.591,Tuesday,False,1,dinner,0,0,0,55.1,1.0,6.5


**Dec 25th (Tuesday), 18:18 - 22:18, 4 hrs, 6 meal steps.**

**Meal Steps Description:**
1. Start:
    - meal
        - potatos (chips?)
    - drink
        - keiths tap, half liter
2. 25 minutes later
    - drink
        - keiths tap, half liter
3. 20 minutes later
    - meal
        - perogies, snack
4. 50 min later
    - drink
        - keiths tap, half liter
5. 25 minutes later
    - drink
        - keiths tap, half liter
6. 30 minutes later
    - drink
        - keiths tap, half liter


**total meals**: 2 snack type foods, **total drinks**: 5 beers, half liters

**`Drinking`** - Dinner time 

`dinner time`, `large amount of alcohol`, `dwell time`, `number of steps` 

**Explanation**: Christams day drinking:(, right at dinner time, eats two somewhat smaller type foods, I wouldnt say they were "meals", more drinking food. The person drinks 5 half liter beers, the first two 25 min apart, and the last three also about 25 min apart.
